# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 14 2022 9:13AM,248617,10,0086074814,ISDIN Corporation,Released
1,Oct 14 2022 9:13AM,248617,10,0086074816,ISDIN Corporation,Released
2,Oct 14 2022 9:13AM,248617,10,0086075599,ISDIN Corporation,Released
3,Oct 14 2022 9:13AM,248617,10,0086075600,ISDIN Corporation,Released
4,Oct 14 2022 9:13AM,248617,10,0086075609,ISDIN Corporation,Released
5,Oct 14 2022 9:13AM,248617,10,0086075608,ISDIN Corporation,Released
6,Oct 14 2022 9:09AM,248616,10,0086074761,ISDIN Corporation,Released
7,Oct 14 2022 9:09AM,248616,10,0086074772,ISDIN Corporation,Released
8,Oct 14 2022 9:09AM,248616,10,0086074768,ISDIN Corporation,Released
9,Oct 14 2022 9:09AM,248616,10,0086074770,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,248612,Released,1
28,248613,Released,17
29,248615,Released,1
30,248616,Released,38
31,248617,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248612,NaN,NaN,1.0
248613,NaN,NaN,17.0
248615,NaN,NaN,1.0
248616,NaN,NaN,38.0
248617,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248096,0.0,0.0,1.0
248101,0.0,0.0,1.0
248103,0.0,0.0,1.0
248109,0.0,0.0,1.0
248532,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248096,0,0,1
248101,0,0,1
248103,0,0,1
248109,0,0,1
248532,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248096,0,0,1
1,248101,0,0,1
2,248103,0,0,1
3,248109,0,0,1
4,248532,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248096,,,1
1,248101,,,1
2,248103,,,1
3,248109,,,1
4,248532,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation
6,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation
44,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.
45,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation
62,Oct 14 2022 8:41AM,248612,10,Bio-PRF
63,Oct 14 2022 8:34AM,248610,10,Bio-PRF
86,Oct 14 2022 8:31AM,248611,16,"Emersa Waterbox, LLC"
87,Oct 14 2022 8:30AM,248594,10,Bio-PRF
121,Oct 14 2022 8:30AM,248597,10,Bio-PRF
146,Oct 14 2022 8:30AM,248604,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation,,,6
1,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation,,,38
2,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.,,,1
3,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation,,,17
4,Oct 14 2022 8:41AM,248612,10,Bio-PRF,,,1
5,Oct 14 2022 8:34AM,248610,10,Bio-PRF,,,23
6,Oct 14 2022 8:31AM,248611,16,"Emersa Waterbox, LLC",,,1
7,Oct 14 2022 8:30AM,248594,10,Bio-PRF,,1,33
8,Oct 14 2022 8:30AM,248597,10,Bio-PRF,1,5,19
9,Oct 14 2022 8:30AM,248604,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation,6,,
1,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation,38,,
2,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.,1,,
3,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation,17,,
4,Oct 14 2022 8:41AM,248612,10,Bio-PRF,1,,
5,Oct 14 2022 8:34AM,248610,10,Bio-PRF,23,,
6,Oct 14 2022 8:31AM,248611,16,"Emersa Waterbox, LLC",1,,
7,Oct 14 2022 8:30AM,248594,10,Bio-PRF,33,1,
8,Oct 14 2022 8:30AM,248597,10,Bio-PRF,19,5,1
9,Oct 14 2022 8:30AM,248604,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation,6,,
1,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation,38,,
2,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.,1,,
3,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation,17,,
4,Oct 14 2022 8:41AM,248612,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation,6.0,NaN,NaN
1,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation,38.0,NaN,NaN
2,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation,17.0,NaN,NaN
4,Oct 14 2022 8:41AM,248612,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 9:13AM,248617,10,ISDIN Corporation,6.0,0.0,0.0
1,Oct 14 2022 9:09AM,248616,10,ISDIN Corporation,38.0,0.0,0.0
2,Oct 14 2022 8:59AM,248615,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Oct 14 2022 8:52AM,248613,10,ISDIN Corporation,17.0,0.0,0.0
4,Oct 14 2022 8:41AM,248612,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3480385,161.0,12.0,9.0
16,248611,1.0,0.0,0.0
19,248532,1.0,0.0,0.0
21,2235289,6.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3480385,161.0,12.0,9.0
1,16,248611,1.0,0.0,0.0
2,19,248532,1.0,0.0,0.0
3,21,2235289,6.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,161.0,12.0,9.0
1,16,1.0,0.0,0.0
2,19,1.0,0.0,0.0
3,21,6.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,161.0
1,16,Released,1.0
2,19,Released,1.0
3,21,Released,6.0
4,10,Executing,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,9.0,0.0,0.0,0.0
Executing,12.0,0.0,0.0,3.0
Released,161.0,1.0,1.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,9.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,3.0
2,Released,161.0,1.0,1.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,9.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,3.0
2,Released,161.0,1.0,1.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()